# Onboard all models  defined in .wf1 files
This notebook takes all .wf1 workfile from the folder wfs\ and transform the models  it to a modelflow model.<br>

The overall process is performed in a Dataclass named:**GrapWfModel**<br> 
Close study of this class can be rewarding, but is outside the need of most users.<br>
The overall structure is: Onboard all model defined in a .wf1 file

The overall structure is:

Eviews is started and the wf1 file is loaded
Some transformations are performed on data.
The model is unlinked
The workspace is saved as a wf2 file. Same name with _modelflow appended
 - Eviews is closed
 - The wf2 file is read as a json file.
 - Relevant objects are extracted.
 - The MFMSA variable is extracted, to be saved in the dumpfile.
 - The equations are transformed and normalized to modelflow format and classified into identities and stochastic
 - Stochastic equations are enriched by add_factor and fixing terms (dummy + fixing value)
 - For Stochastic equations new fitted variables are generated - without add add_factors and dummies.
 - A model to generate fitted variables is created
 - A model to generate add_factors is created.
 - A model encompassing the original equations, the model for fitted variables and for add_factors is created.
 - The data series and scalars are shoveled into a Pandas dataframe
 - Some special series are generated as the expression can not be incorporated into modelflow model specifications
 - The model for fitted values is simulated in the specified timespan
 - The model for add_factors is simulated in the timespan set in MFMSA
 - The data descriptions are extracted into a dictionary.
 - Data descriptions for dummies, fixed values, fitted values and add_factors are derived.
 - Now we have a model and a dataframe with all variables which are needed. b The GrapWfModel instance in general keeps most of the steps so the developer can inspect the the different steps.

The GrapWfModel instance in general keeps most of the steps so the developer can inspect the the different steps.  


- Eviews has to be installed 
- pyevies has to be installed 


In [2]:
from pathlib import Path

from modelclass import model 
from modelgrabwf2 import GrabWfModel
model.widescreen()
model.scroll_off()
latex=1

<IPython.core.display.Javascript object>

In [3]:
 %load_ext autoreload
 %autoreload 2

## List potential files
The workfiles are located in the ```wfs``` folder. <br>
The [pathlib.Path](https://docs.python.org/3/library/pathlib.html) class is used to find all wf1 files

In [4]:
filedict = {f.stem[:3].lower():f for f in Path('wfs').glob('*.wf1')}
for modelname,filename in filedict.items():
    print(f'The model and date for {modelname} is located in this workfile:{filename}')


The model and date for ago is located in this workfile:wfs\agosoln.wf1
The model and date for bol is located in this workfile:wfs\BOLSoln.wf1
The model and date for irq is located in this workfile:wfs\IRQSoln_2040.wf1
The model and date for mda is located in this workfile:wfs\MDAsoln.wf1
The model and date for pak is located in this workfile:wfs\PAKSoln.wf1
The model and date for per is located in this workfile:wfs\PERMOD_damage_asFactor20220322.wf1


## Model specific transformations 
Not all Eviews equations has a direct counterpart in modelflow.<br>
To handle that two types of transformations can be provided for a specific model. 
- **eviews_run_lines** which specifies a list of Eviews command lines to be run. Typical to make some transformation of data. 
- **country_trans** which specifies a list of replacements to be done to the Eviews formulas before they are processed. 

As a convention the variables like this \<modelname>_eviews_run_lines and <modelname>_trans

In [30]:
pak_trans = lambda input : input.replace('- 01*D(','-1*D(')   

ago_trans = lambda  input : input.replace('@MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")','MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD') 
ago_eviews_run_lines = ['smpl @ALL','series MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD = @MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")']

mda_trans = lambda input: input.replace('_MDAsbbrev.@coef(2)','_MDASBBREV_at_COEF_2')         
mda_eviews_run_lines = ['Scalar _MDASBBREV_at_COEF_2 = _MDASBBREV.@COEF(+2)']

## Transform all wf1 files in the folder

In [31]:
filename=
allmodels = {modelname: 
     GrabWfModel(filename, 
                        eviews_run_lines= globals().get(f'{modelname}_eviews_run_lines',[]),
                        country_trans    =   globals().get(f'{modelname}_trans'   ,lambda x : x[:]),
                        make_fitted = True,        # make equatios for fitted values of stocastic equations 
                        do_add_factor_calc=True,   # Calculate the add factors which makes the stocastic equations match    
                        fit_start = 2000,          # Start of calculation of fittet model in baseline (to have some historic values) 
                        fit_end   = None,           # end of calc for fittted model, if None taken from mdmfsa options  
                        disable_progress =True
                        ) 
                  for modelname,filename in filedict.items()}


Reading c:\wb new\Modelflow\working_paper\thebook\content\howto\onboard\eviews\wfs\agosoln.wf1
Assummed model name: AGO
Eviewsline to run :smpl @ALL
Eviewsline to run :series MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD = @MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")
The model: AGO is unlinked 
Writing C:\wb new\Modelflow\working_paper\thebook\content\howto\onboard\eviews\wfs\agosoln_modelflow.wf2
Model name: AGO

Processing the model:AGO
testmodel calculated  
Calculation of add factors for AGO calculated  

Reading c:\wb new\Modelflow\working_paper\thebook\content\howto\onboard\eviews\wfs\MDAsoln.wf1
Assummed model name: MDA
Eviewsline to run :Scalar _MDASBBREV_at_COEF_2 = _MDASBBREV.@COEF(+2)
The model: MDA is unlinked 
Writing C:\wb new\Modelflow\working_paper\thebook\content\howto\onboard\eviews\wfs\MDAsoln_modelflow.wf2
Model name: MDA

Processing the model:MDA
testmodel calculated  
Calculation of add factors for MDA calculated  

Reading c:\wb new\Modelflow\working_paper\thebook\

## Check all the models 

In [32]:
for modelname,cmodel in allmodels.items():
    cmodel.test_model(cmodel.start,cmodel.end,maxerr=100,tol=1,showall=0)   # tol determins the max acceptable absolute difference 

AGO calculated  

Chekking residuals for AGO 2016 to 2060

Variable with residuals above threshold
AGONECONPRVTCN                        , Max difference:  2861.03139496 Max Pct    0.0000106457% It is number    44 in the solveorder and error number 1
AGONEGDIFGOVCN                        , Max difference:    21.01837921 Max Pct    0.0000013130% It is number    55 in the solveorder and error number 2
AGONEGDIFTOTCN                        , Max difference:   177.46922302 Max Pct    0.0000007316% It is number    58 in the solveorder and error number 3
AGONEEXPNOILCN                        , Max difference:    34.74809837 Max Pct    0.0000022911% It is number    63 in the solveorder and error number 4
AGONEEXPGOILCN                        , Max difference:   544.94689941 Max Pct    0.0000026926% It is number    66 in the solveorder and error number 5
AGONEEXPGNFSCN                        , Max difference:   504.25928497 Max Pct    0.0000023486% It is number    67 in the solveorder and erro

## Run the models 
Some models need a different alfa (dampening factor in Gauss iterations) in order to solve. 

In [33]:
alfadict={'ago':0.7}

In [34]:
for modelname,cmodel in allmodels.items():
    _ = cmodel.mmodel(cmodel.base_input,alfa=alfadict.get(modelname,1.0))
    cmodel.mmodel.modeldump(f'modelflowdumps/{modelname}.pcim')  
    print(f'{modelname} run and saved')

ago run and saved
mda run and saved
pak run and saved
per run and saved


In [35]:
!dir modelflowdumps\

 Volume in drive C has no label.
 Volume Serial Number is C2DB-095E

 Directory of c:\wb new\Modelflow\working_paper\thebook\content\howto\onboard\eviews\modelflowdumps

10-06-2022  16:23    <DIR>          .
21-06-2022  10:29    <DIR>          ..
09-06-2022  11:41         1.434.250 ago
21-06-2022  10:30         1.434.181 ago.pcim
09-06-2022  11:41         1.029.618 mda
21-06-2022  10:30         1.029.604 mda.pcim
09-06-2022  11:41         3.386.584 pak
21-06-2022  10:30         3.386.691 pak.pcim
09-06-2022  11:41        11.979.848 per
21-06-2022  10:30        11.980.021 per.pcim
               8 File(s)     35.660.797 bytes
               2 Dir(s)  792.506.646.528 bytes free


## Make some names in the namespace

In [36]:
for modelname,cmodel in allmodels.items():
    thismodel = f'm{modelname}'
    thisbaseline = f'{modelname}_baseline'
    thiscmodel = f'{modelname}_cmodel'
    globals()[thismodel] = cmodel.mmodel
    globals()[thisbaseline]= cmodel.base_input 
    globals()[thiscmodel]= cmodel
    print(f'{thismodel}, {thisbaseline} and {thiscmodel} has been created')    

mago, ago_baseline and ago_cmodel has been created
mmda, mda_baseline and mda_cmodel has been created
mpak, pak_baseline and pak_cmodel has been created
mper, per_baseline and per_cmodel has been created


In [37]:
thissilent = 1
_ = mago(ago_baseline,silent=thissilent,alfa=0.7)
_ = mmda(mda_baseline,silent=thissilent)
_ = mpak(pak_baseline,silent=thissilent)
_ = mper(per_baseline,silent=thissilent)

## From here testing - will be deleted 

## Lets create a list of all variable names

In [38]:
allvar = [varname for cmodel  in allmodels.values() for varname in cmodel.mmodel.allvar.keys()]

In [39]:
allvar = [varname for thismodel in [mago,mmda,mpak,mper] for varname in thismodel.allvar.keys()]

In [40]:
allvar_dict  = {varname:cmodel.mmodel.var_description[varname] for cmodel  in allmodels.values() for varname in cmodel.mmodel.allvar.keys()}

In [41]:
len(allvar)

2872

In [42]:
import fnmatch

In [43]:
fnmatch.filter(allvar, '*NECONGOVTCN')

['AGONECONGOVTCN', 'MDANECONGOVTCN', 'PAKNECONGOVTCN', 'PERNECONGOVTCN']

In [44]:
len(set(allvar))

2763

In [45]:
def getnames(pat,allvar=allvar,allvar_dict=allvar_dict):
    selected = [(varname,allvar_dict[varname]) for varname in fnmatch.filter(allvar_dict.keys(), pat)]
    print(*selected,sep='\n')

### Government consumption

In [46]:
getnames('*NECONGOVTCN')

('AGONECONGOVTCN', 'Nominal Govt Cons')
('MDANECONGOVTCN', 'GDP: Final Consumption: Public Administration LCU thousands')
('PAKNECONGOVTCN', 'Govt. Cons., LCU mn')
('PERNECONGOVTCN', 'PERNECONGOVTCN')


### Government investment 

In [47]:
getnames('*NEGDIFGOVCN')

('AGONEGDIFGOVCN', 'AGONEGDIFGOVCN')
('MDANEGDIFGOVCN', 'MDANEGDIFGOVCN')
('PAKNEGDIFGOVCN', 'Public Fixed Domestic Inv., LCU mn')
('PERNEGDIFGOVCN', 'PERNEGDIFGOVCN')


In [48]:
getnames('*LMEMPTOTLCN*')

('PAKLMEMPTOTLCN', 'Employment')
('PAKLMEMPTOTLCN_FITTED', 'Fitted  value:Employment')
('PAKLMEMPTOTLCN_D', 'Fix dummy:Employment')
('PAKLMEMPTOTLCN_A', 'Add factor:Employment')
('PAKLMEMPTOTLCN_X', 'Fix value:Employment')


In [49]:
getnames('*GGEXPWAGECN*')

('AGOGGEXPWAGECN_A', 'Add factor:Wages and Salaries')
('AGOGGEXPWAGECN_VALUE_2019', 'AGOGGEXPWAGECN_VALUE_2019')
('AGOGGEXPWAGECN_FITTED', 'Fitted  value:Wages and Salaries')
('AGOGGEXPWAGECN', 'Wages and Salaries')
('AGOGGEXPWAGECN_D', 'Fix dummy:Wages and Salaries')
('AGOGGEXPWAGECN_X', 'Fix value:Wages and Salaries')
('MDAGGEXPWAGECN', 'Expenditure: Wages and Salaries LCU millions')
('MDAGGEXPWAGECN_D', 'Fix dummy:Expenditure: Wages and Salaries LCU millions')
('MDAGGEXPWAGECN_X', 'Fix value:Expenditure: Wages and Salaries LCU millions')
('MDAGGEXPWAGECN_FITTED', 'Fitted  value:Expenditure: Wages and Salaries LCU millions')
('MDAGGEXPWAGECN_A', 'Add factor:Expenditure: Wages and Salaries LCU millions')
('PERGGEXPWAGECN_A', 'Add factor:PERGGEXPWAGECN')
('PERGGEXPWAGECN_D', 'Fix dummy:PERGGEXPWAGECN')
('PERGGEXPWAGECN_X', 'Fix value:PERGGEXPWAGECN')
('PERGGEXPWAGECN', 'PERGGEXPWAGECN')
('PERGGEXPWAGECN_FITTED', 'Fitted  value:PERGGEXPWAGECN')


In [50]:
mpak.PAKBXFSTREMTCD.frml

Endogeneous: PAKBXFSTREMTCD: Exp., Remittances (BOP), US$ mn
Formular: FRML <Z,EXO> PAKBXFSTREMTCD = (PAKBXFSTREMTCD_A*PAKNYGDPMKTPCD+PAKNYGDPMKTPCD* (0.053028373687723) ) * (1-PAKBXFSTREMTCD_D)+ PAKBXFSTREMTCD_X*PAKBXFSTREMTCD_D  $

PAKBXFSTREMTCD  : Exp., Remittances (BOP), US$ mn
PAKBXFSTREMTCD_A: Add factor:Exp., Remittances (BOP), US$ mn
PAKBXFSTREMTCD_D: Fix dummy:Exp., Remittances (BOP), US$ mn
PAKBXFSTREMTCD_X: Fix value:Exp., Remittances (BOP), US$ mn
PAKNYGDPMKTPCD  : 




In [51]:
mpak.PAKGGEXPGNFSCN.frml

Endogeneous: PAKGGEXPGNFSCN: General government expenditure on goods and services (millions lcu)
Formular: FRML <Z,EXO> PAKGGEXPGNFSCN = (0.3*PAKGGEXPGNFSCN(-1)+(1-0.3)*(0.80*(PAKGGREVTOTLCN-PAKGGEXPINTPCN(-0)))+505277.259999999*DURING_2012 + PAKGGEXPGNFSCN_A)* (1-PAKGGEXPGNFSCN_D)+ PAKGGEXPGNFSCN_X*PAKGGEXPGNFSCN_D  $

PAKGGEXPGNFSCN  : General government expenditure on goods and services (millions lcu)
DURING_2012     : 
PAKGGEXPGNFSCN_A: Add factor:General government expenditure on goods and services (millions lcu)
PAKGGEXPGNFSCN_D: Fix dummy:General government expenditure on goods and services (millions lcu)
PAKGGEXPGNFSCN_X: Fix value:General government expenditure on goods and services (millions lcu)
PAKGGEXPINTPCN  : General government interest payments on public debt (millions lcu)
PAKGGREVTOTLCN  : General government revenues, total (millions lcu)




In [52]:
###  mpak.modeldash('PAKNECONGOVTCN',jupyter=1)

In [53]:
allmodels['pak'].all_frml_dict['PAKGGEXPGNFSCN']

Endo_var        : PAKGGEXPGNFSCN
Original        : PAKGGEXPGNFSCN=0.3*PAKGGEXPGNFSCN(-1)+(1-0.3)*(0.80*(PAKGGREVTOTLCN-PAKGGEXPINTPCN(-0)))+505277.259999999*DURING_2012
Preprocessed    : PAKGGEXPGNFSCN=0.3*PAKGGEXPGNFSCN(-1)+(1-0.3)*(0.80*(PAKGGREVTOTLCN-PAKGGEXPINTPCN(-0)))+505277.259999999*DURING_2012
Normalized      : PAKGGEXPGNFSCN = (0.3*PAKGGEXPGNFSCN(-1)+(1-0.3)*(0.80*(PAKGGREVTOTLCN-PAKGGEXPINTPCN(-0)))+505277.259999999*DURING_2012 + PAKGGEXPGNFSCN_A)* (1-PAKGGEXPGNFSCN_D)+ PAKGGEXPGNFSCN_X*PAKGGEXPGNFSCN_D 
Calc_add_factor : PAKGGEXPGNFSCN_A = (PAKGGEXPGNFSCN) - (0.3*PAKGGEXPGNFSCN(-1)+(1-0.3)*(0.80*(PAKGGREVTOTLCN-PAKGGEXPINTPCN(-0)))+505277.259999999*DURING_2012)
Fitted          : PAKGGEXPGNFSCN_FITTED = 0.3*PAKGGEXPGNFSCN(-1)+(1-0.3)*(0.80*(PAKGGREVTOTLCN-PAKGGEXPINTPCN(-0)))+505277.259999999*DURING_2012
Eviews          : PAKGGEXPGNFSCN = 0.3*PAKGGEXPGNFSCN( - 1) + (1 - 0.3)*(0.80*(PAKGGREVTOTLCN - PAKGGEXPINTPCN( - 0))) + 505277.259999999*@DURING("2012")


In [54]:
allmodels['ago'].all_frml_dict['AGONECONGOVTCN']

Endo_var        : AGONECONGOVTCN
Original        : DIFF(AGONECONGOVTCN)=DIFF(AGOGGEXPGNFSCN+AGOGGEXPWAGECN)*AGONECONGOVTCOV_/100+14591.8357729705*DUMH
Preprocessed    : ((AGONECONGOVTCN)-(AGONECONGOVTCN(-1)))=((AGOGGEXPGNFSCN+AGOGGEXPWAGECN)-(AGOGGEXPGNFSCN(-1)+AGOGGEXPWAGECN(-1)))*AGONECONGOVTCOV_/100+14591.8357729705*DUMH
Normalized      : AGONECONGOVTCN = (AGONECONGOVTCN_A+ (((AGOGGEXPGNFSCN+AGOGGEXPWAGECN)-(AGOGGEXPGNFSCN(-1)+AGOGGEXPWAGECN(-1)))*AGONECONGOVTCOV_/100+14591.8357729705*DUMH) +AGONECONGOVTCN(-1)) * (1-AGONECONGOVTCN_D)+ AGONECONGOVTCN_X*AGONECONGOVTCN_D 
Calc_add_factor : AGONECONGOVTCN_A = AGONECONGOVTCN- ((((AGOGGEXPGNFSCN+AGOGGEXPWAGECN)-(AGOGGEXPGNFSCN(-1)+AGOGGEXPWAGECN(-1)))*AGONECONGOVTCOV_/100+14591.8357729705*DUMH)) -AGONECONGOVTCN(-1)
Fitted          : AGONECONGOVTCN_FITTED =  (((AGOGGEXPGNFSCN+AGOGGEXPWAGECN)-(AGOGGEXPGNFSCN(-1)+AGOGGEXPWAGECN(-1)))*AGONECONGOVTCOV_/100+14591.8357729705*DUMH) +AGONECONGOVTCN(-1)
Eviews          : D(AGONECONGOVTCN) = D(AGOGG

In [55]:
allmodels['pak'].mmodel

<
Model name                              :                  PAK 
Model structure                         :         Simultaneous 
Number of variables                     :                  841 
Number of exogeneous  variables         :                  463 
Number of endogeneous variables         :                  378 
>